# BACK TESTING

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
export_path = 'io/output/exports/'
predictions_path = export_path + 'predictions/'
gcn_list = ['A3TGCN2', 'DCRNN', 'TGCN', 'TGCN2', 'GConvGRU', 'GConvLSTM', 'GCLSTM']
gcn = gcn_list[3] #TGCN2

In [ ]:
pivot_true = pd.read_csv(predictions_path+gcn+'_predictions.csv',index_col=0)
pivot_pred = pd.read_csv(predictions_path+gcn+'_ground_truth.csv',index_col=0)

In [ ]:
# Define trading strategy
def trading_strategy(predicted_close, current_close):
    if predicted_close > current_close + (current_close * 0.2):
        return 'buy'
    elif predicted_close < current_close + (current_close * 0.2):
        return 'sell'
    else:
        return 'hold'

# Initialize variables
balance = 1000.0
shares = 0.0
trades = []
results = []

# Loop through each day in the data
for i in range(len(pivot_true)-1):
    # Get current close price
    current_close = pivot_true['DASH'][i]
    
    # Use neural network to predict next close price
    predicted_close = pivot_pred['DASH'][i]
    
    # Determine trading action based on strategy
    action = trading_strategy(predicted_close, current_close)
    
    # Execute trade
    if action == 'buy':
        shares = balance / current_close
        balance = 0.0
        trades.append(('buy', current_close))
    elif action == 'sell':
        balance = shares * current_close
        shares = 0.0
        trades.append(('sell', current_close))
    else:
        trades.append(('hold', current_close))
    
    # Record results
    results.append(balance + shares * current_close)

print(trades)
# Calculate performance metrics
profit = results[-1] - results[0]
return_rate = profit / results[0] * 100.0
win_rate = sum([1 for t in trades if t[0] == 'buy' and t[1] < t[1:][0]]) / len([t for t in trades if t[0] == 'buy']) * 100.0

# Plot results
plt.plot(results)
plt.xlabel('Day')
plt.ylabel('Balance')
plt.title('Backtest Results')
plt.show()

print('Profit: ${:.2f}'.format(profit))
print('Return rate: {:.2f}%'.format(return_rate))
print('Win rate: {:.2f}%'.format(win_rate))

